In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Web3 Security Researcher AI Assistant

**Capstone Project for 5-day Gen AI Intensive Course with Google (Apr 2025)**

**Goal:** To create an AI assistant capable of analyzing Solidity smart contracts for common security vulnerabilities and gas optimization opportunities, leveraging Google's Gemini model, custom analysis tools, and Retrieval Augmented Generation (RAG) for enhanced context.

**Approach:** This notebook utilizes the direct `google-generativeai` Python library for interaction with the Gemini Pro model (`models/gemini-1.5-pro-latest`). This approach was chosen for stability after encountering persistent hanging issues with the `langchain-google-genai` wrapper in the Kaggle environment during development.

The assistant works by:
1.  Running basic pattern-matching tools (`VulnerabilityAnalysisTool`, `GasOptimizerTool`) on the input Solidity code.
2.  Using Retrieval Augmented Generation (RAG) to find descriptions of known vulnerabilities similar to the input code/findings.
3.  Feeding the original code, tool outputs, and RAG results into the Gemini model via a structured prompt.
4.  Asking the Gemini model to act as a Web3 Security Researcher and synthesize a comprehensive analysis, including explanations of risks and recommendations.

**Gen AI Capabilities Demonstrated:**
* **Function Calling:** Defining and using specialized tools (`VulnerabilityAnalysisTool`, `GasOptimizerTool`) to perform specific analysis tasks.
* **Embeddings:** Generating vector embeddings for a knowledge base of vulnerability descriptions using Google's embedding model.
* **Retrieval Augmented Generation (RAG):** Retrieving relevant known vulnerability information based on input code similarity to enhance the LLM's context.
* **Vector Search/Similarity:** Using cosine similarity to find the most relevant items from the embedded knowledge base.

**Requires:** A Google AI API Key stored as a Kaggle Secret named `GOOGLE_API_KEY`.

In [2]:
# Install necessary packages
# Using google-generativeai for direct calls, langchain-core for BaseTool structure
!pip install google-generativeai pandas numpy langchain-core scikit-learn # Added scikit-learn for RAG similarity
# Optional: For potential future embedding alternatives or vector stores
# !pip install sentence-transformers faiss-cpu
# Optional: For UI
# !pip install ipywidgets

# Upgrade core Google libraries (might help with compatibility)
# Note: This might produce dependency conflict warnings with other pre-installed Kaggle packages.
# These warnings can often be ignored unless they cause direct errors later.
!pip install --upgrade google-api-core google-auth google-generativeai

print("--- Installation and Upgrades Complete ---")
print("!!! IMPORTANT: Please RESTART the Kernel/Session now !!!")
print("(Use Runtime -> Restart session, or the restart button in the toolbar)")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 74.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.70.0
    Uninstalling grpcio-1.70.0:
      Successfully uninstalled grpcio-1.70.0
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.27.0
    Uninstalling google-auth-2.27.0:
      Successfully uninstalled google-auth-2.27.0
  Attempting uninstall: grpcio-status
    Found existing installation: grpcio-status 1.48.2
    Uninstalling 

In [3]:
import os
import google.generativeai as genai
import pandas as pd
import numpy as np
import json
from kaggle_secrets import UserSecretsClient
import warnings

# Suppress specific warnings if needed (optional)
warnings.filterwarnings("ignore", category=UserWarning, module="langchain_google_genai")
warnings.filterwarnings("ignore", category=UserWarning, module="langsmith")


# --- Workaround for Kaggle environment ---
# Attempt to disable GCE metadata check which can cause hangs/errors
os.environ['NO_GCE_CHECK'] = 'True'
# Attempt to disable ADC environment lookup
os.environ['GOOGLE_AUTH_DISABLE_CREDENTIALS_ENV'] = 'True'
# Ensure GOOGLE_APPLICATION_CREDENTIALS is not set
os.environ.pop('GOOGLE_APPLICATION_CREDENTIALS', None)
print("Environment workarounds set (NO_GCE_CHECK=True).")
# --- End Workaround ---

# Optional: Disable LangSmith tracing explicitly
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_API_KEY"] = ""

# Configure Google API Key
try:
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("GOOGLE_API_KEY")
    genai.configure(api_key=api_key)
    print("Google AI SDK Configured successfully.")
except Exception as e:
    print(f"API Key configuration failed. Make sure 'GOOGLE_API_KEY' is set in Kaggle Secrets.")
    print(f"Error: {e}")
    # Raise the error to prevent proceeding without a key
    raise e

# Define the model name we confirmed works from list_models
# If needed, run genai.list_models() to confirm availability
WORKING_MODEL_NAME = 'models/gemini-1.5-pro-latest'
# Define the embedding model name
EMBEDDING_MODEL_NAME = 'models/embedding-001'

print(f"Using LLM: {WORKING_MODEL_NAME}")
print(f"Using Embedding Model: {EMBEDDING_MODEL_NAME}")

Environment workarounds set (NO_GCE_CHECK=True).
Google AI SDK Configured successfully.
Using LLM: models/gemini-1.5-pro-latest
Using Embedding Model: models/embedding-001


In [4]:
# Sample vulnerability database for RAG context
# In a real application, this could be much larger and loaded from a file/database
vulnerability_db = [
    {
        "name": "Reentrancy",
        "description": "A critical vulnerability where an external call from a contract back into itself (or another contract that calls back) before the initial function completes allows state manipulation, often leading to draining funds. It exploits the default gas forwarding of low-level calls like .call(). Mitigation involves the Checks-Effects-Interactions pattern or mutex/reentrancy guards.",
        "keywords": "call.value, external call, state change order, mutex, reentrancy guard, DAO hack"
    },
    {
        "name": "Integer Overflow/Underflow",
        "description": "Occurs when an arithmetic operation results in a value exceeding the maximum or going below the minimum storage capacity of the variable type (e.g., uint256). Before Solidity 0.8.0, this would 'wrap around', potentially leading to incorrect calculations for token balances, prices, or array indices. Mitigation involves using SafeMath libraries (pre-0.8.0) or relying on Solidity 0.8.0+'s default checked arithmetic.",
        "keywords": "uint, SafeMath, arithmetic, wrap around, maximum value, minimum value, solidity 0.8"
    },
    {
        "name": "Access Control / Authorization",
        "description": "Flaws related to improper or missing checks on who is allowed to execute sensitive functions (e.g., changing ownership, withdrawing funds, pausing the contract). Can allow unauthorized users to take control or misuse contract functionality. Mitigation involves using modifiers like 'onlyOwner', role-based access control (RBAC), and verifying msg.sender appropriately.",
        "keywords": "access control, authorization, onlyOwner, require msg.sender, private, internal, external, RBAC, Parity hack"
    },
    {
        "name": "Unchecked Return Values",
        "description": "Ignoring the success/failure boolean returned by low-level calls like .call(), .delegatecall(), .staticcall(), or even some external contract calls like ERC20's transfer(). If the call fails silently (returns false), the calling contract might proceed assuming success, leading to inconsistent state or loss of funds. Mitigation requires checking the boolean return value and handling potential failures, often with a require() statement.",
        "keywords": "call, delegatecall, staticcall, send, transfer, return value, bool success, require success"
    },
    {
        "name": "Gas Limit Issues / Denial of Service (DoS)",
        "description": "Vulnerabilities where an attacker can prevent legitimate users from using the contract or parts of it. Can occur via 'gas griefing' (making operations too expensive), block stuffing, or designing functions that rely on unbounded loops or external calls that can fail or run out of gas. Mitigation involves capping iterations, using pull-over-push patterns for payments, and avoiding reliance on potentially failing external calls for core logic.",
        "keywords": "DoS, denial of service, gas limit, out of gas, unbounded loop, pull over push"
    },
    {
        "name": "Timestamp Dependence",
        "description": "Using block.timestamp as a source of randomness or for critical timing logic. Miners have some control over timestamps, making them manipulable within limits. This can affect lottery outcomes, vesting schedules, or condition checks based on time. Mitigation involves avoiding direct reliance on block.timestamp for sensitive operations or using block numbers combined with reasonable time estimates.",
        "keywords": "block.timestamp, timestamp manipulation, randomness, timing attack, miner influence"
    },
    {
        "name": "Front-Running",
        "description": "An attack where an observer (often a miner or bot) sees a pending transaction in the mempool and submits their own transaction with a higher gas price to get executed first, potentially taking advantage of the information or state change in the original transaction (e.g., DEX trades, NFT mints). Mitigation can involve commit-reveal schemes, batching transactions, or designing mechanisms less sensitive to transaction order.",
        "keywords": "front-running, mempool, transaction ordering, gas price, DEX arbitrage, MEV"
    }
    # Add more vulnerabilities as needed...
]

# Convert to DataFrame
vulnerability_df = pd.DataFrame(vulnerability_db)

# Combine name, description, keywords for embedding
vulnerability_df['text_for_embedding'] = vulnerability_df['name'] + ": " + vulnerability_df['description'] + " Keywords: " + vulnerability_df['keywords']

print(f"Vulnerability Knowledge Base created with {len(vulnerability_df)} entries.")
vulnerability_df.head()

Vulnerability Knowledge Base created with 7 entries.


,name,description,keywords,text_for_embedding
0,Reentrancy,A critical vulnerability where an external cal...,"call.value, external call, state change order,...",Reentrancy: A critical vulnerability where an ...
1,Integer Overflow/Underflow,Occurs when an arithmetic operation results in...,"uint, SafeMath, arithmetic, wrap around, maxim...",Integer Overflow/Underflow: Occurs when an ari...
2,Access Control / Authorization,Flaws related to improper or missing checks on...,"access control, authorization, onlyOwner, requ...",Access Control / Authorization: Flaws related ...
3,Unchecked Return Values,Ignoring the success/failure boolean returned ...,"call, delegatecall, staticcall, send, transfer...",Unchecked Return Values: Ignoring the success/...
4,Gas Limit Issues / Denial of Service (DoS),Vulnerabilities where an attacker can prevent ...,"DoS, denial of service, gas limit, out of gas,...",Gas Limit Issues / Denial of Service (DoS): Vu...


In [5]:
import google.generativeai as genai
import numpy as np
import time # For potential retry delays

# --- Global variables for RAG ---
description_embeddings = None
embeddings_object = None
# --------------------------------

# --- Function to generate embeddings with retries ---
def generate_embeddings_with_retry(model_name, content_list, task_type, retries=3, delay=5):
    for attempt in range(retries):
        try:
            response = genai.embed_content(
                model=model_name,
                content=content_list,
                task_type=task_type
            )
            return np.array(response['embedding'])
        except Exception as e:
            print(f"Embedding attempt {attempt+1}/{retries} failed: {e}")
            if attempt < retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print("Max retries reached. Failed to generate embeddings.")
                raise e # Re-raise the last exception

# --- Generate embeddings for the knowledge base ---
try:
    print(f"Generating embeddings for Knowledge Base using {EMBEDDING_MODEL_NAME}...")
    texts_to_embed = vulnerability_df['text_for_embedding'].tolist()

    # Generate embeddings for the documents in the knowledge base
    description_embeddings = generate_embeddings_with_retry(
        EMBEDDING_MODEL_NAME,
        texts_to_embed,
        "RETRIEVAL_DOCUMENT" # Use RETRIEVAL_DOCUMENT for the DB items
    )

    if description_embeddings is not None:
        print(f"Generated {description_embeddings.shape[0]} embeddings with dimension {description_embeddings.shape[1]}.")

        # Define a simple wrapper class or function to handle query embedding
        # This provides the '.embed_query()' method expected by the retrieval function
        class SimpleEmbedderWrapper:
             def __init__(self, model_name):
                 self.model_name = model_name
             def embed_query(self, query):
                 print(f"Embedding query using {self.model_name}...")
                 # Use RETRIEVAL_QUERY for the user query / search input
                 return generate_embeddings_with_retry(self.model_name, [query], "RETRIEVAL_QUERY")[0] # Embed one query

        embeddings_object = SimpleEmbedderWrapper(EMBEDDING_MODEL_NAME)
        print("Embeddings object ready for retrieval function.")
    else:
        print("Embedding generation failed after retries.")

except Exception as e:
    print(f"An error occurred during the embedding process: {e}")
    # Ensure variables are None if process failed
    description_embeddings = None
    embeddings_object = None

Generating embeddings for Knowledge Base using models/embedding-001...
Generated 7 embeddings with dimension 768.
Embeddings object ready for retrieval function.


In [6]:
from sklearn.metrics.pairwise import cosine_similarity # Make sure scikit-learn is installed

def retrieve_similar_vulnerabilities(query: str, top_k: int = 3):
    """
    Retrieves the top_k most similar vulnerability descriptions from the
    knowledge base based on cosine similarity of embeddings.
    """
    global embeddings_object, description_embeddings, vulnerability_df # Use global vars

    if embeddings_object is None or description_embeddings is None:
        print("Embeddings not available, skipping RAG.")
        return []

    try:
        # Create embedding for the query
        query_embedding = embeddings_object.embed_query(query)
        query_embedding = np.array([query_embedding]) # Reshape for cosine_similarity

        # Calculate cosine similarity
        similarities = cosine_similarity(query_embedding, description_embeddings)[0]

        # Get indices of top k most similar items (descending order)
        # Argsort gives ascending, so we take the last 'top_k' and reverse them
        top_indices = np.argsort(similarities)[-top_k:][::-1]

        # Return the corresponding entries from the DataFrame
        results = []
        print(f"RAG Top {top_k} matches (Indices: {top_indices}, Similarities: {similarities[top_indices]}):")
        for idx in top_indices:
            results.append(vulnerability_df.iloc[idx].to_dict())
            print(f"- Found: {vulnerability_df.iloc[idx]['name']}")

        return results

    except Exception as e:
        print(f"Error during similarity search: {e}")
        return []

print("RAG retrieval function defined.")

RAG retrieval function defined.


In [7]:
from typing import Type, List
from pydantic import BaseModel, Field
from langchain_core.tools import BaseTool
import json

# --- Input Schema (defined again for clarity if needed, or rely on previous cell) ---
class ContractCodeInput(BaseModel):
    code: str = Field(description="Solidity smart contract code to analyze")

# --- Vulnerability Tool (Updated to use RAG) ---
class VulnerabilityAnalysisTool(BaseTool):
    name: str = "vulnerability_analyzer"
    description: str = "Analyzes Solidity code for common security vulnerabilities using pattern matching and retrieves similar known issues."
    args_schema: Type[BaseModel] = ContractCodeInput

    def _run(self, code: str) -> str:
        print(f"\n--- Running {self.name} ---")
        vulnerabilities = []
        # Reentrancy Check
        if "call.value" in code and "ReentrancyGuard" not in code:
            print("Pattern matched: Potential Reentrancy")
            vulnerabilities.append({
                "type": "Reentrancy", "severity": "High",
                "details": "Potential reentrancy vulnerability detected due to call.value without explicit guards. State changes should ideally happen before external calls (Checks-Effects-Interactions)."
            })
        # Unchecked Return Value Check
        if ".call(" in code and "require" not in code and ".transfer(" not in code and ".send(" not in code:
            if "bool success" not in code and "require(success" not in code:
                 print("Pattern matched: Potential Unchecked Return Value")
                 vulnerabilities.append({
                    "type": "Unchecked Return Value", "severity": "Medium",
                    "details": "Low-level calls (.call, .delegatecall, .staticcall) return a boolean indicating success. This value should be checked."
                 })

        # --- RAG Call ---
        context_results = []
        print("Attempting RAG...")
        # Create a query for RAG based on detected issues or code snippet
        rag_query = f"Source Code Snippet (first 500 chars): {code[:500]}"
        if vulnerabilities:
             rag_query += f"\nDetected potential issues: {[v['type'] for v in vulnerabilities]}"

        context_results = retrieve_similar_vulnerabilities(rag_query, top_k=2) # Find top 2 similar
        # ----------------

        print(f"Tool results - Detected: {len(vulnerabilities)}, RAG Context: {len(context_results)}")
        return json.dumps({
            "detected_issues": vulnerabilities,
            "similar_known_vulnerabilities": context_results # Include RAG results
         }, indent=2) # Add indent for readability if printed

# --- Gas Optimizer Tool ---
class GasOptimizerTool(BaseTool):
    name: str = "gas_optimizer"
    description: str = "Analyzes Solidity code for potential gas optimization opportunities using basic checks."
    args_schema: Type[BaseModel] = ContractCodeInput

    def _run(self, code: str) -> str:
        print(f"\n--- Running {self.name} ---")
        optimizations = []
        # Storage Packing Suggestion
        if "uint" in code and "storage" in code:
             print("Pattern matched: Potential Storage Packing")
             optimizations.append({
                "type": "Storage Packing",
                "description": "Consider packing multiple smaller variables (e.g., uint128, bool, address) into single 256-bit storage slots if they are declared consecutively.",
                "impact": "Can reduce gas costs for storage reads and writes."
            })
        # Loop Optimization Suggestion
        if "for (" in code and "++" in code and "pragma solidity ^0.8" in code:
             print("Pattern matched: Potential Loop Optimization")
             optimizations.append({
                "type": "Loop Optimization",
                "description": "For simple increments/decrements within loops (e.g., i++) where overflow/underflow is impossible or acceptable, consider using an 'unchecked' block (Solidity >=0.8.0).",
                "impact": "Can save gas on arithmetic operations within loops with many iterations."
            })
        print(f"Tool results - Optimizations found: {len(optimizations)}")
        return json.dumps({"optimizations": optimizations}, indent=2)

# Instantiate tools
vuln_tool = VulnerabilityAnalysisTool()
gas_tool = GasOptimizerTool()

print("Security analysis tools defined (updated with RAG call).")

Security analysis tools defined (updated with RAG call).


In [8]:
import google.generativeai as genai
import json # Ensure json is imported

# System prompt defining the AI's persona and task
AGENT_SYSTEM_PROMPT = """You are a specialized Web3 Security Researcher AI Assistant. Your goal is to provide a clear and helpful security analysis of Solidity smart contracts.

You analyze the provided contract code along with findings from basic analysis tools and context from potentially similar known vulnerabilities (RAG). Your analysis should cover:
- **Identified Vulnerabilities:** Explain potential vulnerabilities detected by the tools OR your own analysis (like Reentrancy, Unchecked Return Values, Access Control issues, Integer Overflow/Underflow etc.). Refer to the 'Tool: Vulnerability Analysis' section for tool findings and 'Tool: RAG Context' for similar known issues. Cite specific code lines/snippets where possible. Explain the risks associated with each vulnerability.
- **Gas Optimizations:** Mention potential gas saving opportunities identified by the 'Tool: Gas Optimization' section.
- **Overall Assessment:** Provide a brief summary of the contract's security posture based on the analysis.
- **Recommendations:** Suggest concrete steps or patterns to mitigate the identified vulnerabilities and improve security.

Be methodical and thorough. Explain concepts clearly for developers. Structure your final response clearly.
"""

# Initialize the working Gemini model directly (check if already done in Cell 2/3, or redo here)
try:
    # Check if gemini_model already exists from a previous cell, otherwise create it
    if 'gemini_model' not in locals() or gemini_model is None:
         print("Initializing Gemini model...")
         gemini_model = genai.GenerativeModel(WORKING_MODEL_NAME)
         print(f"Direct Gemini model ({WORKING_MODEL_NAME}) initialized.")
    else:
         print(f"Using existing Gemini model ({WORKING_MODEL_NAME}).")
except Exception as e:
    print(f"Failed to initialize/confirm Gemini model: {e}")
    gemini_model = None

# Function to perform analysis using direct LLM calls
def analyze_contract_directly(contract_code: str):
    if not gemini_model:
        return "Error: Gemini model not initialized."

    print("\n--- Starting Direct Analysis Pipeline ---")
    # 1. Run Tools
    print("Step 1: Running analysis tools...")
    try:
        vuln_results_json_str = vuln_tool._run(code=contract_code)
        gas_results_json_str = gas_tool._run(code=contract_code)
        # Parse JSON results from tools
        vuln_results = json.loads(vuln_results_json_str)
        gas_results = json.loads(gas_results_json_str)
        print("Tools finished.")
    except Exception as e:
        print(f"Error running tools: {e}")
        return f"Error running analysis tools: {e}"

    # 2. Construct the Prompt for the LLM
    print("Step 2: Constructing prompt for LLM...")
    prompt = f"{AGENT_SYSTEM_PROMPT}\n\n"
    prompt += "--- Contract Code Under Analysis ---\n```solidity\n" + contract_code + "\n```\n\n"
    prompt += "--- Tool Output: Vulnerability Analysis ---\n"
    prompt += f"```json\n{json.dumps(vuln_results, indent=2)}\n```\n\n" # Include full tool output
    prompt += "--- Tool Output: Gas Optimization ---\n"
    prompt += f"```json\n{json.dumps(gas_results, indent=2)}\n```\n\n" # Include full tool output
    prompt += "--- Your Analysis Task ---\n"
    prompt += "Based *only* on the provided contract code and the tool outputs above (especially the detected issues and RAG context), please provide your comprehensive security analysis, overall assessment, and concrete recommendations."

    # 3. Call the LLM
    print(f"Step 3: Calling Gemini model ({WORKING_MODEL_NAME})...")
    try:
        # Configure generation - you can adjust temperature, top_p, etc.
        generation_config = genai.types.GenerationConfig(
            # candidate_count=1, # default is 1
            # stop_sequences=["..."],
            # max_output_tokens=2048, # Default is often large enough
            temperature=0.3 # Slightly creative but still focused
        )
        # Set safety settings to be less restrictive if needed, but be careful
        safety_settings = {
            'HARASSMENT': 'BLOCK_MEDIUM_AND_ABOVE',
            'HATE_SPEECH': 'BLOCK_MEDIUM_AND_ABOVE',
            'SEXUAL': 'BLOCK_MEDIUM_AND_ABOVE',
            'DANGEROUS': 'BLOCK_MEDIUM_AND_ABOVE'
        }

        response = gemini_model.generate_content(
            prompt,
            generation_config=generation_config,
            safety_settings=safety_settings
        )
        print("LLM call complete.")

        # Check for blocked response due to safety or other reasons
        if not response.candidates:
             print("Warning: LLM response was empty or blocked.")
             try:
                 # Try to access prompt feedback if available
                 block_reason = response.prompt_feedback.block_reason
                 safety_ratings = response.prompt_feedback.safety_ratings
                 print(f"Block Reason: {block_reason}")
                 print(f"Safety Ratings: {safety_ratings}")
                 return f"Error: LLM response blocked. Reason: {block_reason}. Ratings: {safety_ratings}"
             except Exception:
                 return "Error: LLM response was empty or blocked for an unknown reason."

        # Access the text, handling potential multipart response if vision model was used accidentally
        if hasattr(response, 'text'):
            final_analysis = response.text
        elif response.candidates and hasattr(response.candidates[0].content, 'parts'):
             final_analysis = "".join(part.text for part in response.candidates[0].content.parts)
        else:
             final_analysis = "Error: Could not extract text from LLM response."
             print(f"Unexpected response structure: {response}")

        return final_analysis

    except Exception as e:
        print(f"Error during LLM generation: {e}")
        # Attempt to access potentially more detailed error information if available
        error_details = getattr(e, 'message', str(e))
        return f"Error generating analysis from LLM: {error_details}"

print("Direct analysis function defined.")

Initializing Gemini model...
Direct Gemini model (models/gemini-1.5-pro-latest) initialized.
Direct analysis function defined.


In [9]:
# Sample vulnerable contract
sample_contract = """
pragma solidity ^0.8.0;

contract VulnerableBank {
    mapping(address => uint) private balances;

    function deposit() public payable {
        balances[msg.sender] += msg.value;
    }

    // Vulnerable withdraw function
    function withdraw() public {
        uint amount = balances[msg.sender];
        // Unchecked call before state change -> Reentrancy risk
        (bool success, ) = msg.sender.call{value: amount}("");
        // Missing check for 'success' -> Unchecked Return Value risk
        balances[msg.sender] = 0; // State change after external call
    }

    function getBalance() public view returns (uint) {
        return balances[msg.sender];
    }
}
"""

print("--- Running Direct Analysis on Sample Contract ---")
# Make sure previous cells defining the function and tools have been run
analysis_result = analyze_contract_directly(sample_contract)

print("\n✅ --- Analysis Result --- ✅")
# Print the final result from the LLM
from IPython.display import display, Markdown # For nice formatting
display(Markdown(analysis_result))
print("--------------------------")

--- Running Direct Analysis on Sample Contract ---

--- Starting Direct Analysis Pipeline ---
Step 1: Running analysis tools...

--- Running vulnerability_analyzer ---
Attempting RAG...
Embedding query using models/embedding-001...
RAG Top 2 matches (Indices: [3 2], Similarities: [0.69454917 0.69295947]):
- Found: Unchecked Return Values
- Found: Access Control / Authorization
Tool results - Detected: 0, RAG Context: 2

--- Running gas_optimizer ---
Tool results - Optimizations found: 0
Tools finished.
Step 2: Constructing prompt for LLM...
Step 3: Calling Gemini model (models/gemini-1.5-pro-latest)...
LLM call complete.

✅ --- Analysis Result --- ✅


## Security Analysis of VulnerableBank Contract

This report details the security analysis of the `VulnerableBank` contract based on the provided code and tool outputs.

**Identified Vulnerabilities:**

1. **Reentrancy:** The `withdraw()` function is susceptible to reentrancy attacks.  The external call `msg.sender.call{value: amount}("")` is made *before* updating the user's balance (`balances[msg.sender] = 0`).  A malicious contract could call back into the `withdraw()` function within the external call, repeatedly draining funds before the balance is zeroed.  This is a critical vulnerability.
    * **Code Snippet:**
    ```solidity
    function withdraw() public {
        uint amount = balances[msg.sender];
        (bool success, ) = msg.sender.call{value: amount}(""); // External call before state update
        balances[msg.sender] = 0; // State update after external call
    }
    ```
    * **Risk:** Complete loss of funds in the contract.

2. **Unchecked Return Value:** The `withdraw()` function does not check the return value of the low-level call `msg.sender.call{value: amount}("")`. The `success` boolean is ignored. If the call fails for any reason (e.g., the recipient contract reverts), the `withdraw()` function proceeds as if the transfer was successful, setting the user's balance to zero even though the funds were not transferred. This can lead to inconsistencies in the contract's state and potential loss of user funds.
    * **Code Snippet:**
    ```solidity
    (bool success, ) = msg.sender.call{value: amount}(""); // success is not checked
    ```
    * **Risk:** Loss of user funds if the external call fails.


**Gas Optimizations:**

No gas optimization opportunities were identified by the tool. However, using the `Checks-Effects-Interactions` pattern in the `withdraw` function (updating the state *before* making the external call) inherently reduces gas costs associated with potential reentrancy mitigations like reentrancy guards.


**Overall Assessment:**

The contract has critical security vulnerabilities, primarily the reentrancy vulnerability in the `withdraw()` function, making it highly insecure. The unchecked return value further exacerbates the risk.  Deployment in its current state is strongly discouraged.


**Recommendations:**

1. **Mitigate Reentrancy:** Implement the Checks-Effects-Interactions pattern. Update the user's balance *before* making the external call:
    ```solidity
    function withdraw() public {
        uint amount = balances[msg.sender];
        balances[msg.sender] = 0; // State update before external call
        (bool success, ) = msg.sender.call{value: amount}(""); // External call
        require(success, "Transfer failed."); // Check return value
    }
    ```
2. **Check Return Values:** Always check the `success` boolean returned by low-level calls like `.call()`.  Use `require(success, "Transfer failed.");` to revert the transaction if the call fails. This is already included in the code snippet above.
3. **Consider Reentrancy Guards:** While the Checks-Effects-Interactions pattern is preferred, consider using a reentrancy guard (e.g., a boolean flag or a mutex) as an additional layer of protection, especially in complex contracts.
4. **Thorough Testing:** Conduct extensive testing, including unit tests and fuzzing, to identify and address potential vulnerabilities before deployment.  Specifically, test the `withdraw()` function with a malicious contract that attempts a reentrancy attack.


By addressing these issues, the security posture of the `VulnerableBank` contract can be significantly improved.


--------------------------


---
## Appendix: Notes on LangChain Agent Attempt

Initially, this project attempted to use a LangChain ReAct Agent (`langchain.agents.create_react_agent` with `ChatGoogleGenerativeAI`). However, during development and testing within the Kaggle Notebook environment (as of April 2025), persistent issues were encountered where calls to the LangChain agent executor (`agent_executor.invoke`) or direct calls to the LangChain LLM wrapper (`llm.invoke`) would hang indefinitely.

This occurred despite:
* Using confirmed valid API keys.
* Using model names confirmed available via `genai.list_models()`.
* Applying environment variable workarounds (`NO_GCE_CHECK=True`, etc.) intended to mitigate issues related to Kaggle's restricted environment potentially interfering with Google library metadata checks.
* Confirming that direct calls using the base `google-generativeai` library *were* successful with the same setup.

The traceback during hangs pointed to waits deep within the underlying `grpc` library's network communication layer. This suggests a potential incompatibility or bug within the `langchain-google-genai` wrapper's handling of API calls or authentication flows specifically within this Kaggle environment.

Due to these unresolved issues and the project deadline, the primary implementation was switched to use the **direct `google-generativeai` library calls**, orchestrated by the `analyze_contract_directly` function defined above. This approach proved stable and functional. The LangChain agent code cells have been removed from the main flow to ensure notebook runnability for the Capstone submission.

In [10]:
# --- Optional: Interactive UI ---
# This UI calls the reliable 'analyze_contract_directly' function.

from IPython.display import display, HTML, clear_output, Markdown
import ipywidgets as widgets

print("Setting up UI widgets...")

# Create text area for contract input
contract_input_ui = widgets.Textarea(
    value=sample_contract, # Pre-fill with sample
    placeholder='Paste your Solidity contract here',
    description='Contract:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='300px')
)

# Create button for analysis
analyze_button_ui = widgets.Button(
    description='Analyze Contract (Direct Method)',
    disabled=False,
    button_style='success', # Use success style for the working method
    tooltip='Click to analyze the contract using the direct Gemini call',
    icon='check-circle'
)

# Output area using Markdown for better formatting
output_ui = widgets.Output()
# Display Markdown within the output area requires this workaround sometimes
html_output = widgets.HTML(value="")
output_container = widgets.VBox([output_ui, html_output])


# Callback function for button click
def on_button_clicked_ui(b):
    # Link button press to output area and clear previous
    with output_ui:
        clear_output(wait=True) # Clear previous output nicely
        html_output.value = "" # Clear HTML output area too
        print("Analysis requested...")

        contract_code = contract_input_ui.value
        if not contract_code:
            print("Please enter a contract to analyze.")
            return

        # Call the reliable direct analysis function
        print("Analyzing contract using direct method...")
        analysis_result_direct = analyze_contract_directly(contract_code)

        print("\n--- Analysis Complete ---")
        # Display result using Markdown for better formatting directly in HTML widget
        html_output.value = Markdown(analysis_result_direct).data


# Connect the button to callback
analyze_button_ui.on_click(on_button_clicked_ui)

# Display the UI
print("Displaying UI (run this cell last if using UI)...")
display(contract_input_ui, analyze_button_ui, output_container)

Setting up UI widgets...
Displaying UI (run this cell last if using UI)...


Textarea(value='\npragma solidity ^0.8.0;\n\ncontract VulnerableBank {\n    mapping(address => uint) private b…

Button(button_style='success', description='Analyze Contract (Direct Method)', icon='check-circle', style=Butt…

---
# Conclusion

This notebook demonstrated the creation of a Web3 Security Researcher AI Assistant using Google's Gemini Pro model. Key capabilities included:

* **Direct LLM Interaction:** Successfully used the `google-generativeai` library to analyze Solidity code based on a detailed system prompt and contextual information.
* **Function Calling Simulation:** Implemented custom Python tools (`VulnerabilityAnalysisTool`, `GasOptimizerTool`) using `langchain-core`'s `BaseTool` structure, simulating function calling by running these tools directly and feeding their structured output (JSON) into the LLM prompt.
* **RAG Implementation:** Created a knowledge base of vulnerabilities, generated embeddings using `models/embedding-001`, performed similarity searches, and included retrieved context in the LLM prompt to enhance analysis accuracy.

**Challenges & Workarounds:**
Significant challenges were encountered with the `langchain-google-genai` library wrapper within the Kaggle environment, leading to persistent hangs during API calls. Workarounds like setting `NO_GCE_CHECK=True` resolved issues for direct library calls but not for the LangChain wrapper. Consequently, the primary analysis pathway uses direct SDK calls.

**Future Work:**
* Resolve LangChain wrapper compatibility issues (possibly requiring library updates or reporting bugs).
* Expand the vulnerability knowledge base and RAG capabilities.
* Integrate more sophisticated static analysis techniques (e.g., Slither output) as tool inputs.
* Develop more comprehensive evaluation metrics.
* Explore integration with on-chain data via blockchain explorers.